In [6]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to the SQLite database
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Books (
        BookID INTEGER PRIMARY KEY,
        Title TEXT,
        Genre TEXT,
        ISBN INTEGER,
        PublicationYear INTEGER,
        Stock INTEGER,
        ShelfLocation TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Members (
        MemberID INTEGER PRIMARY KEY,
        Name TEXT,
        MembershipType TEXT,
        JoinDate TEXT,
        ActiveStatus TEXT,
        TotalLoans INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Roles (
        RoleID INTEGER PRIMARY KEY,
        RoleName TEXT,
        RoleLevel INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Staff (
        StaffID INTEGER PRIMARY KEY,
        Name TEXT,
        RoleID INTEGER,
        HireDate TEXT,
        ActiveStatus TEXT,
        FOREIGN KEY (RoleID) REFERENCES Roles(RoleID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Loans (
        LoanID INTEGER PRIMARY KEY,
        MemberID INTEGER,
        BookID INTEGER,
        LoanDate TEXT,
        DueDate TEXT,
        ReturnDate TEXT,
        LateFees REAL,
        FOREIGN KEY (MemberID) REFERENCES Members(MemberID),
        FOREIGN KEY (BookID) REFERENCES Books(BookID)
    )
''')

# Function to generate random strings for names and titles
def random_string(length):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(random.choices(letters, k=length))

# Function to generate random names
def random_name():
    first_names = ['John', 'Jane', 'Alex', 'Emily', 'Chris', 'Kate', 'Daniel', 'Laura', 'Michael', 'Sarah']
    last_names = ['Smith', 'Johnson', 'Brown', 'Taylor', 'Anderson', 'Thomas', 'Moore', 'Wilson', 'Clark', 'Lewis']
    return f"{random.choice(first_names)} {random.choice(last_names)}"

# Function to generate random dates
def random_date(start_year=2000, end_year=2023):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    random_days = random.randint(0, (end_date - start_date).days)
    return (start_date + timedelta(days=random_days)).strftime('%Y-%m-%d')

# Populate Books table
genres = ["Fiction", "Non-Fiction", "Science", "History", "Mystery"]
print("Populating Books...")
for i in range(1, 1001):
    title = f"Book-{random_string(5)}"
    genre = random.choice(genres)
    isbn = random.randint(1000000000, 9999999999)
    pub_year = random.randint(1950, 2023)
    stock = random.randint(1, 20)
    shelf_location = f"Shelf-{random.randint(1, 100)}"
    # Removing BookID from the insert statement as it's auto-incrementing
    cursor.execute("INSERT INTO Books (Title, Genre, ISBN, PublicationYear, Stock, ShelfLocation) VALUES (?, ?, ?, ?, ?, ?)",
                   (title, genre, isbn, pub_year, stock, shelf_location))

# Populate Members table
membership_types = ["Standard", "Premium", "VIP"]
print("Populating Members...")
for i in range(1, 1001):
    name = random_name()
    membership = random.choice(membership_types)
    join_date = random_date(2000, 2023)
    active_status = random.choice(["Active", "Inactive"])
    total_loans = random.randint(0, 50)
    cursor.execute("INSERT INTO Members VALUES (?, ?, ?, ?, ?, ?)",
                   (i, name, membership, join_date, active_status, total_loans))

# Populate Roles table
roles = [("Librarian", 1), ("Assistant Librarian", 2), ("Senior Staff", 3), ("Junior Staff", 4), ("Volunteer", 5)]
print("Populating Roles...")
for i, (role_name, role_level) in enumerate(roles, 1):
    cursor.execute("INSERT INTO Roles VALUES (?, ?, ?)", (i, role_name, role_level))

# Populate Staff table
print("Populating Staff...")
for i in range(1, 101):
    name = random_name()
    role_id = random.randint(1, len(roles))  # Assign random role from Roles table
    hire_date = random_date(2000, 2023)
    active_status = random.choice(["Active", "Inactive"])
    cursor.execute("INSERT INTO Staff VALUES (?, ?, ?, ?, ?)", (i, name, role_id, hire_date, active_status))

# Populate Loans table
print("Populating Loans...")
for i in range(1, 1001):
    member_id = random.randint(1, 1000)  # MemberID from Members table
    book_id = random.randint(1, 1000)    # BookID from Books table
    loan_date = random_date(2018, 2023)
    due_date = (datetime.strptime(loan_date, '%Y-%m-%d') + timedelta(days=random.randint(7, 30))).strftime('%Y-%m-%d')
    return_date = random.choice([None, random_date(2018, 2023)])
    late_fees = 0
    if return_date and return_date > due_date:
        late_fees = round(random.uniform(1, 50), 2)
    cursor.execute("INSERT INTO Loans VALUES (?, ?, ?, ?, ?, ?, ?)",
                   (i, member_id, book_id, loan_date, due_date, return_date, late_fees))

# Commit changes and close connection
conn.commit()
conn.close()
print("Database populated successfully!")

Populating Books...
Populating Members...
Populating Roles...
Populating Staff...
Populating Loans...
Database populated successfully!
